In [2]:
library(SPARQL)


Loading required package: XML
Loading required package: RCurl
Loading required package: bitops


In [4]:

val group = z.input("group", "")
z.put("group", group)
val name = z.input("name","Reference Designator")
z.put("name", name)


ERROR: Error in parse(text = x, srcfile = src): <text>:2:5: unexpected symbol
1: 
2: val group
       ^


In [ ]:
%spark

val fuseki = z.input("fuseki", "https://imce-caesar-repository.jpl.nasa.gov/")
z.put("fuseki", fuseki)

val dataset = z.input("dataset")
z.put("dataset", dataset)


In [ ]:
%spark.r
endpoint <- "http://localhost:3030/WBv6/query"

query <-
'
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX fse: <http://imce.jpl.nasa.gov/discipline/fse/fse#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX base: <http://imce.jpl.nasa.gov/foundation/base/base#>
PREFIX mission: <http://imce.jpl.nasa.gov/foundation/mission/mission#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX EuropaTypes: <http://europa.jpl.nasa.gov/efse/libraries/EuropaTypes#>
select distinct ?subSystemNumber  ?subSystemRefDes ?subSystemLabel ?subSystemAcronym ?provenanceSubsystem ?assemblyAcronym ?assemblyName ?assemblyRefDes ?parentAssemblyRefDes ?mountingLocation ?provenanceAssembly 
where {
  #Getting Subsystem
    graph ?gs {?ss a ?st}
    ?gs fse:hasProvenance ?provenanceSubsystem .
    ?st rdfs:subClassOf fse:Subsystem . 
    ?ss fse:hasSubsystemAcronym ?subSystemAcronym ;
        fse:hasSubsystemNumber ?subSystemNumber ;
        fse:hasSubsystemReferenceDesignator ?subSystemRefDes ;
        base:hasAlternateName ?subSystemLabel .
   
  
   [ base:hasAggregatesSource ?ss; 
      base:hasAggregatesTarget ?a ].
  
  #Getting Assembly
    graph ?ga {?a a ?at } .
    ?ga fse:hasProvenance ?provenanceAssembly .
    ?at rdfs:subClassOf fse:Assembly .
    ?a fse:hasAssemblyReferenceDesignator ?assemblyRefDes ;
       base:hasAlternateName ?assemblyAcronym ;
       base:hasCanonicalName ?assemblyName .
  optional{
                ?parenta a base:Contains ;
            base:hasContainsSource ?pa;
               base:hasContainsTarget ?a.
        ?pa fse:hasAssemblyReferenceDesignator ?parentAssemblyRefDes ;
  }
  optional{
    ?a EuropaTypes:BasicAssembly.Mounting_Location ?mountingLocation.
  }
      

}


'
qd <- SPARQL(endpoint,query)
df <- qd$results


In [ ]:
%spark.r

#Defining Column Names
colNames<- c("SUBSYSTEM_NUMBER" , "SUBSYSTEM_REFDES", "SUBSYSTEM_NAME", "SUBSYSTEM_ACRONYM", "PROVENANCE_SUBSYSTEM", "ASSEMBLY_ACRONYM", "ASSEMBLY_NAME" , "ASSEMBLY_REFDES","PARENT_ASSEMBLY", "MOUNTING_LOCATION", "PROVENANCE_ASSEMBLY")

#Aggregating multiple function numbers into a coma separated single row
d <- tryCatch(
    {
        #Renaming column Names
        names(df) <-colNames
        createDataFrame(sqlContext,df)
        
    }, error=function(e) {
    
        #Creating Empty data frame
        emptydf <- data.frame(rbind(sapply(colNames,function(x)NA)))
        createDataFrame(sqlContext,emptydf)
        
    })
    

registerTempTable (d, "EFSE_REF_DES")




In [ ]:
%sql
SELECT distinct  PROVENANCE_SUBSYSTEM as Configuration_Management_State, SUBSYSTEM_NUMBER as Subsystem_Number,   SUBSYSTEM_ACRONYM as Subsystem_Or_Assembly_Acronym , SUBSYSTEM_NAME as Subsystem_Or_Assembly_Name, SUBSYSTEM_REFDES as Reference_Designator, "-" as Parent_Assembly, "-" as Mounting_Location FROM EFSE_REF_DES  union all (select PROVENANCE_ASSEMBLY, SUBSYSTEM_REFDES,  ASSEMBLY_ACRONYM, ASSEMBLY_NAME, ASSEMBLY_REFDES,PARENT_ASSEMBLY, MOUNTING_LOCATION  from EFSE_REF_DES)
